In [1]:
import twint
import csv
import pandas as pd
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, EmotionOptions
import os

In [5]:
df = pd.read_csv('/Users/amogh/Documents/Study/twitter/datasets/raw/mumbai_unlock1.csv')

In [11]:
df.columns

Index(['id', 'conversation_id', 'created_at', 'date', 'time', 'timezone',
       'user_id', 'username', 'name', 'place', 'tweet', 'mentions', 'urls',
       'photos', 'replies_count', 'retweets_count', 'likes_count', 'hashtags',
       'cashtags', 'link', 'retweet', 'quote_url', 'video', 'near', 'geo',
       'source', 'user_rt_id', 'user_rt', 'retweet_id', 'reply_to',
       'retweet_date', 'translate', 'trans_src', 'trans_dest'],
      dtype='object')

In [ ]:
#df = pd.DataFrame([[1, 1.5]], columns=['int', 'float'])
#row = next(df.iterrows())[1]

In [51]:
for index, row in df[['tweet', 'link']][:10].iterrows():
    print(index, ''.join(filter(str.isalpha, row['tweet'])))

0 MumbaicoronavirusupdatesMissionBeginAgainUnlockSuspectedPositiveTotalpositivepatientsdeathsdischargedtodaydeathsinthecitysinceTeststillJunepictwittercomfiRdajDQ
1 SeeingthecurrentCOVIDIndiasituationandtalkingabtunlockinMumbaiissurprisingNewcaseshavenotstoppedNoothercitywldhavedreamedtoevenUnlockincurrentsituationMumbaishdhavebeenlastinCountrytoopenupOfficeofUTPMOIndianarendramodiin
2 AmappreparedbyCSIRNEERIMumbaishowingCOVIDーcasesinIndiathJunecoronavirusMaharashtraFightsCoronaMedicalMaharashtraMEDDcoronaupdatesindiaUnlockingIndiaUnlockMumbaiFightsCovidStayHomeStaySafeSaveLivesCOVIDIndiaUnlockIndiapictwittercomuRrmiUoiPv
3 IthinkprobablyusyoungpeopleintheofficehavealreadycontactthevirusbutareasymptomaticsaidafriendwhohasbeenworkingoutofherresearchlabsinceMarchunlockMumbai
4 MissionBeginAgainMumbaiCrowdssurgeonDayofUnlockshouldgovernmentstaggertimingsAsTuesdaysawlongqueuesattrainstationsexpertstellstatetoconsidervariedofficehourshttpsmmiddaycomamparticlesmumbaicrowdssurgeondayofunlocks

In [66]:
for index, row in df[['tweet', 'link']][:1].iterrows():
    #print(row['tweet'])
    stripped_tweet = ''
    for char in row['tweet']:
        if str.isalpha(char) or str.isspace(char):
            stripped_tweet += char
    print(stripped_tweet)

Mumbai coronavirus updates

MissionBeginAgain
Unlock
   Suspected Positive
  Total positive patients
  deaths 
  discharged today
   deaths in the city since  

Tests till  June   pictwittercomfiRdajDQ


In [3]:
ibm_api = os.environ.get("ibm_api")
ibm_url = os.environ.get("ibm_url")

In [4]:
authenticator = IAMAuthenticator(ibm_api)
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2019-07-12',
    authenticator=authenticator
)

natural_language_understanding.set_service_url(ibm_url)

In [39]:
response = natural_language_understanding.analyze(
    text="I hate apples",
    features=Features(emotion=EmotionOptions(targets=['apples']))).get_result()

In [40]:
print(json.dumps(response, indent=2))

{
  "usage": {
    "text_units": 1,
    "text_characters": 13,
    "features": 1
  },
  "language": "en",
  "emotion": {
    "targets": [
      {
        "text": "apples",
        "emotion": {
          "sadness": 0.107092,
          "joy": 0.005216,
          "fear": 0.127117,
          "disgust": 0.242079,
          "anger": 0.774741
        }
      }
    ],
    "document": {
      "emotion": {
        "sadness": 0.107092,
        "joy": 0.005216,
        "fear": 0.127117,
        "disgust": 0.242079,
        "anger": 0.774741
      }
    }
  }
}


In [5]:
def clean_tweet(tweet):
    stripped_tweet = ''
    for char in tweet:
        if str.isalpha(char) or str.isspace(char):
            stripped_tweet += char.lower()
    return stripped_tweet

In [101]:
clean_tweet('helloS')

'hellos'

In [6]:
def analyze(tweet, target):
    response = natural_language_understanding.analyze(text=tweet,features=Features(emotion=EmotionOptions(targets=[target]))).get_result()
    return response['emotion']['document']['emotion']

In [78]:
mango = analyze('i love mango')

In [86]:
output = pd.DataFrame()
mango['link'] = 'google.com'
output = output.append(mango, ignore_index=True)
output = output.append(mango, ignore_index=True)

In [87]:
output

,anger,disgust,fear,joy,link,sadness
0,0.774741,0.242079,0.127117,0.005216,google.com,0.107092
1,0.774741,0.242079,0.127117,0.005216,google.com,0.107092


In [130]:
output = pd.DataFrame()

for index, row in df[['tweet', 'link']].iterrows():
    if(index % 20 == 0):
        print("At Index: {}".format(index))
    tweet = row['tweet']
    stripped_tweet = clean_tweet(tweet)
    try:
        result = analyze(stripped_tweet, target='mumbai')
        result['link'] = row['link']
        output = output.append(result, ignore_index=True)
    except:
        pass

ion-id: d0575229e48bd1e10ccc3791371f3a4c
At Index: 120
ERROR:root:unsupported text language: hi
Traceback (most recent call last):
  File "/opt/miniconda3/envs/twitter/lib/python3.8/site-packages/ibm_cloud_sdk_core/base_service.py", line 228, in send
    raise ApiException(
ibm_cloud_sdk_core.api_exception.ApiException: Error: unsupported text language: hi, Code: 400 , X-global-transaction-id: 141b522f0710b3fba62a9a9bf2040337
ERROR:root:cannot locate keyphrase
Traceback (most recent call last):
  File "/opt/miniconda3/envs/twitter/lib/python3.8/site-packages/ibm_cloud_sdk_core/base_service.py", line 228, in send
    raise ApiException(
ibm_cloud_sdk_core.api_exception.ApiException: Error: cannot locate keyphrase, Code: 400 , X-global-transaction-id: 78456106c38848fbdb693cfd9506d22c
ERROR:root:unsupported text language: hi
Traceback (most recent call last):
  File "/opt/miniconda3/envs/twitter/lib/python3.8/site-packages/ibm_cloud_sdk_core/base_service.py", line 228, in send
    raise A

In [131]:
column_names = ["link", "anger", "disgust", "fear", "joy", "sadness"]

apple = output.reindex(columns=column_names)

In [132]:
apple.to_csv('lol.csv', index=False, encoding='utf-8')

In [121]:
for index, row in df[['tweet', 'link']][:5].iterrows():
    tweet = row['tweet']
    print(tweet)
    print(tweet.split("\n"))

Mumbai #coronavirus updates.
17.06.2020
#MissionBeginAgain
#Unlock1
🗣️ 716  Suspected Positive
🤒 1359 Total positive patients
☠️ 77 deaths, 
🏃 298 discharged today
☠️ 3242  deaths in the city since 25.01.2020 

Tests till 16 June 2,69,608  pic.twitter.com/fiR9dajD7Q
['Mumbai #coronavirus updates.', '17.06.2020', '#MissionBeginAgain', '#Unlock1', '🗣️ 716  Suspected Positive', '🤒 1359 Total positive patients', '☠️ 77 deaths, ', '🏃 298 discharged today', '☠️ 3242  deaths in the city since 25.01.2020 ', '', 'Tests till 16 June 2,69,608  pic.twitter.com/fiR9dajD7Q']
Seeing the current #COVID19India situation and talking abt #unlock2  in #Mumbai is surprising! New cases have not stopped! No other city wld have dreamed to even #Unlock1 in current situation.Mumbai shd have been last in Country to open up! @OfficeofUT @PMOIndia @narendramodi_in
['Seeing the current #COVID19India situation and talking abt #unlock2  in #Mumbai is surprising! New cases have not stopped! No other city wld have drea

In [9]:
df = pd.read_csv('/Users/amogh/Documents/Study/twitter/datasets/raw/kolkata_unlock1.csv')

output = pd.DataFrame()

for index, row in df[['tweet', 'link']].iterrows():
    if(index % 20 == 0):
        print("At Index: {}".format(index))
    tweet = row['tweet']
    stripped_tweet = clean_tweet(tweet)
    try:
        result = analyze(stripped_tweet, target='kolkata')
        result['link'] = row['link']
        output = output.append(result, ignore_index=True)
    except:
        pass

column_names = ["link", "anger", "disgust", "fear", "joy", "sadness"]

apple = output.reindex(columns=column_names)

apple.to_csv('kolkata.csv', index=False, encoding='utf-8')

At Index: 0
ERROR:root:unsupported text language: bn
Traceback (most recent call last):
  File "/opt/miniconda3/envs/twitter/lib/python3.8/site-packages/ibm_cloud_sdk_core/base_service.py", line 228, in send
    raise ApiException(
ibm_cloud_sdk_core.api_exception.ApiException: Error: unsupported text language: bn, Code: 400 , X-global-transaction-id: 12c0e7ed2112a2ebee0645efebbd03f3
ERROR:root:unsupported text language: unknown
Traceback (most recent call last):
  File "/opt/miniconda3/envs/twitter/lib/python3.8/site-packages/ibm_cloud_sdk_core/base_service.py", line 228, in send
    raise ApiException(
ibm_cloud_sdk_core.api_exception.ApiException: Error: unsupported text language: unknown, Code: 400 , X-global-transaction-id: 15faefece91730a18cd65a7f30698e20
ERROR:root:cannot locate keyphrase
Traceback (most recent call last):
  File "/opt/miniconda3/envs/twitter/lib/python3.8/site-packages/ibm_cloud_sdk_core/base_service.py", line 228, in send
    raise ApiException(
ibm_cloud_sdk_